In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ranji\\My Projects\\Projects\\Kidney-Disease-Classification-Deep-Learning-MLOps-Pipeline\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ranji\\My Projects\\Projects\\Kidney-Disease-Classification-Deep-Learning-MLOps-Pipeline'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_classes: int
    params_weights: str
    params_include_top: bool
    params_learning_rate: float

In [6]:
from src.KidneyDiseaseClassifier.constants import *
from src.KidneyDiseaseClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "ct-kidney-images-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_classes=params.CLASSES,
            params_weights=params.WEIGHTS,
            params_include_top=params.INCLUDE_TOP,
            params_learning_rate=float(params.LEARNING_RATE)
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [10]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        # Load the saved updated base model (.keras)
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
        #self.model.summary()

        # Compile the model again after loading
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        path = str(path)
        if not path.endswith(".keras"):
            path += ".keras"
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # Add callbacks
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)
        ]
        
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callbacks
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )



In [11]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e


[2025-07-12 14:55:20,740: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-12 14:55:20,742: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-12 14:55:20,743: INFO: common: created directory at: artifacts]
[2025-07-12 14:55:20,744: INFO: common: created directory at: artifacts\training]
[2025-07-12 14:55:22,820: WARNING: hdf5_format: No training configuration found in the save file, so the model was *not* compiled. Compile it manually.]
Found 400 images belonging to 4 classes.
Found 1600 images belonging to 4 classes.
Epoch 1/10
100/100 [==============================] - 94s 890ms/step - loss: 1.9995 - accuracy: 0.3363 - val_loss: 1.6328 - val_accuracy: 0.1800 - lr: 1.0000e-04
Epoch 2/10
100/100 [==============================] - 92s 925ms/step - loss: 1.4491 - accuracy: 0.4831 - val_loss: 2.9505 - val_accuracy: 0.2850 - lr: 1.0000e-04
Epoch 3/10
100/100 [==============================] - 85s 849ms/step - loss: 1.1009 - accuracy: 0.5875 - val_l